In [1]:
import pandas as pd
import polars as pl
import polars.selectors as cs
import numpy as np
import plotnine as pn
import plotly.graph_objs as go
import plotly.express as px
from tqdm.notebook import tqdm
from IPython.display import clear_output, display
import os
from itertools import product

from deap import base, creator, tools, algorithms
import imageio


%load_ext blackcellmagic

### Loading the Price Data

In [2]:
df = pd.read_csv("../01 - Data/example_week.csv")
df.head()

,spot,utc_time
0,101.54,2022-01-01 00:00:00+00:00
1,52.13,2022-01-01 01:00:00+00:00
2,20.78,2022-01-01 02:00:00+00:00
3,15.66,2022-01-01 03:00:00+00:00
4,21.47,2022-01-01 04:00:00+00:00


### The Power Plant

In [3]:
plant_params = {
    "EFFICIENCY": 0.75,
    "MAX_STORAGE_M3": 5000,
    "MIN_STORAGE_M3": 0,
    "TURBINE_POWER_MW": 100,
    "PUMP_POWER_MW": 100,
    "TURBINE_RATE_M3H": 500,
    "MIN_STORAGE_MWH": 0,
    "INITIAL_WATER_LEVEL_PCT": 0,
}
plant_params["INITIAL_WATER_LEVEL"] = (
    plant_params["INITIAL_WATER_LEVEL_PCT"] * plant_params["MAX_STORAGE_M3"]
)
plant_params["PUMP_RATE_M3H"] = (
    plant_params["TURBINE_RATE_M3H"] * plant_params["EFFICIENCY"]
)
plant_params["MAX_STORAGE_MWH"] = (
    plant_params["MAX_STORAGE_M3"] / plant_params["TURBINE_RATE_M3H"]
) * plant_params["TURBINE_POWER_MW"]

### Using Genetic Algorithms with DEAP: Understanding the Library

Since the actual structure of the required individuals in genetic algorithms does strongly depend on the task at hand, DEAP does not contain any explicit structure. It will rather provide a convenient method for creating containers of attributes, associated with fitnesses, called the deap.creator. Using this method we can create custom individuals in a very simple way.

In [4]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

The creator is a class factory that can build new classes at run-time. It will be called with first the desired name of the new class, second the base class it will inherit, and in addition any subsequent arguments you want to become attributes of your class.

In [5]:
ind_size = df.shape[0]

All the objects we will use on our way, an individual, the population, as well as all functions, operators, and arguments will be stored in a DEAP container called Toolbox. It contains two methods for adding and removing content, register() and unregister().

In [6]:
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_action", np.random.choice, [-1, 0, 1])

Initialise a generation function that samples uniformly in the input space.

In [7]:
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_action, ind_size)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [8]:
# CXPB  is the probability with which two individuals
# are crossed
# MUTPB is the probability for mutating an individual
CXPB, MUTPB = 0.5, 0.1

In [9]:
revenues = np.select(
    condlist=[
        np.array(toolbox.individual()) == -1,
        np.array(toolbox.individual()) == 1,
    ],
    choicelist=[
        -plant_params["PUMP_POWER_MW"] * df["spot"],
        plant_params["TURBINE_POWER_MW"] * df["spot"],
    ],
    default=0,
)
revenues

array([-10154.,   5213.,      0.,   1566.,      0.,   2989.,      0.,
        -9536.,      0.,      0.,      0.,  -7072.,  -7499.,  -8746.,
            0.,      0.,  14889.,      0., -13776.,      0.,  14062.,
        14449., -14177.,  -8851.,  -7504.,  -4538.,   2370.,   1584.,
            0.,   3588.,  -4900.,  -7505.,      0.,   7592.,      0.,
         7504.,  -4876.,  -5825.,      0.,      0., -12625.,      0.,
            0.,      0.,   8978.,      0.,  -8738.,      0.,      0.,
         1668.,      0.,      0.,   7860., -14186., -19940.,      0.,
       -21573.,      0.,  20634.,      0., -19196.,      0.,      0.,
       -22446.,      0., -21411.,      0.,      0.,      0.,      0.,
        14655.,      0.,  11197.,      0.,  -9775.,      0., -12272.,
            0., -19784.,      0.,  21280.,  20799., -20426., -19130.,
            0., -19350.,      0.,      0.,      0., -20580.,      0.,
            0.,      0., -15284., -13574., -14144., -11712.,      0.,
        -9261.,  -91

In [10]:
water_levels = np.select(
    condlist=[
        np.array(toolbox.individual()) == -1,
        np.array(toolbox.individual()) == 1,
    ],
    choicelist=[plant_params["PUMP_RATE_M3H"], plant_params["TURBINE_RATE_M3H"]],
    default=0,
).cumsum()

exceedances = (
    (water_levels >= plant_params["MAX_STORAGE_M3"])
    | (water_levels <= plant_params["MIN_STORAGE_M3"])
).sum()

In [11]:
# Set the evaluation function
def evaluate_fitness(individual):

    # Calculate revenues from actions
    revenues = np.select(
        condlist=[
            np.array(individual) == -1,
            np.array(individual) == 1,
        ],
        choicelist=[
            -plant_params["PUMP_POWER_MW"] * df["spot"],
            plant_params["TURBINE_POWER_MW"] * df["spot"],
        ],
        default=0,
    )

    # Calculate water level exceedances
    water_levels = np.select(
        condlist=[
            np.array(individual) == -1,
            np.array(individual) == 1,
        ],
        choicelist=[plant_params["PUMP_RATE_M3H"], -plant_params["TURBINE_RATE_M3H"]],
        default=0,
    ).cumsum()

    exceedances = (
        (water_levels >= plant_params["MAX_STORAGE_M3"])
        | (water_levels <= plant_params["MIN_STORAGE_M3"])
    ).sum()

    if exceedances > 0:
        return revenues.sum() - 1e7,
    else:
        return revenues.sum(),
    # always need to return a stupid tuple, otherwise get a completely
    # unhelpful error message and try to find the problem for hours

In [43]:
# Genetic Operators
toolbox.register("evaluate", evaluate_fitness)
toolbox.register("mate", tools.cxUniform, indpb=0.5)
toolbox.register("mutate", tools.mutUniformInt, low=-1, up=1, indpb=0.05)
toolbox.register("select", tools.selBest, fit_attr="fitness")

In [44]:
# Creating popluation
population = toolbox.population(n=50)

In [45]:
toolbox.select(population, k=10)

[[-1,
  0,
  -1,
  1,
  1,
  -1,
  1,
  1,
  1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  1,
  1,
  0,
  0,
  -1,
  -1,
  -1,
  1,
  -1,
  1,
  -1,
  1,
  -1,
  0,
  0,
  -1,
  1,
  -1,
  1,
  1,
  1,
  -1,
  -1,
  -1,
  1,
  0,
  -1,
  0,
  1,
  1,
  1,
  -1,
  -1,
  0,
  1,
  0,
  0,
  -1,
  -1,
  1,
  0,
  1,
  0,
  -1,
  -1,
  1,
  1,
  -1,
  1,
  0,
  -1,
  0,
  1,
  -1,
  -1,
  1,
  0,
  -1,
  0,
  0,
  1,
  -1,
  1,
  1,
  1,
  -1,
  1,
  -1,
  0,
  0,
  -1,
  -1,
  -1,
  1,
  0,
  1,
  -1,
  -1,
  -1,
  1,
  -1,
  -1,
  1,
  -1,
  -1,
  1,
  -1,
  1,
  0,
  -1,
  0,
  1,
  -1,
  1,
  1,
  -1,
  1,
  1,
  -1,
  0,
  -1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  -1,
  1,
  0,
  1,
  -1,
  -1,
  0,
  -1,
  0,
  1,
  0,
  0,
  1,
  -1,
  -1,
  -1,
  1,
  0,
  1,
  -1,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  -1,
  1,
  -1,
  -1,
  0,
  1,
  -1,
  0,
  0,
  1,
  -1,
  0,
  -1,
  -1,
  -1,
  -1],
 [0,
  0,
  0,
  -1,
  0,
  0,
  -1,
  0,
  -1,
  0,
  -1,
  0,
  1,
  -1,
  0,
  -1,
  -1,

In [68]:
toolbox.mate(population[0], population[1])

([-1,
  0,
  -1,
  1,
  1,
  -1,
  1,
  1,
  -1,
  0,
  -1,
  -1,
  1,
  -1,
  0,
  -1,
  1,
  0,
  0,
  1,
  -1,
  -1,
  1,
  -1,
  0,
  -1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  -1,
  -1,
  1,
  0,
  -1,
  1,
  -1,
  0,
  1,
  -1,
  -1,
  0,
  1,
  0,
  0,
  1,
  -1,
  1,
  1,
  1,
  0,
  -1,
  0,
  1,
  0,
  -1,
  -1,
  0,
  -1,
  0,
  1,
  -1,
  -1,
  1,
  0,
  -1,
  0,
  -1,
  1,
  1,
  1,
  0,
  1,
  -1,
  0,
  -1,
  0,
  0,
  -1,
  -1,
  0,
  -1,
  -1,
  1,
  -1,
  -1,
  -1,
  0,
  -1,
  -1,
  -1,
  0,
  1,
  1,
  -1,
  1,
  0,
  -1,
  -1,
  -1,
  -1,
  1,
  1,
  -1,
  -1,
  -1,
  1,
  -1,
  -1,
  1,
  1,
  0,
  0,
  0,
  0,
  -1,
  -1,
  -1,
  0,
  -1,
  0,
  -1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  -1,
  0,
  -1,
  1,
  0,
  1,
  -1,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  -1,
  1,
  1,
  -1,
  -1,
  1,
  -1,
  0,
  0,
  1,
  -1,
  0,
  1,
  -1,
  1,
  -1],
 [0,
  0,
  0,
  -1,
  0,
  0,
  -1,
  0,
  1,
  -1,
  -1,
  0,
  -1,
  -1,
  -1,
  1,
  -1,
  

In [ ]:
population[0]

In [192]:
# Variable keeping track of the number of generations
generation = 0
total_generations = 100
min_values = []
mean_values = []
max_values = []

# Begin the evolution
while generation < total_generations:
    
    # Select the next generation individuals
    offspring = toolbox.select(population, len(population))

    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if np.random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if np.random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    # Those are the ones that have been deleted
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # Keep score of values for evaluation after training
    fit = [ind.fitness.values[0] for ind in offspring]
    min_values.append(np.min(fit))
    mean_values.append(np.mean(fit))
    max_values.append(np.max(fit))

    # Overwrite the current population with the offspring    
    population[:] = offspring

    # Increment counter
    generation += 1
    # Print status
    if generation % 10 == 0:
        print(f"Generation {generation} done...")

Generation 10 done...
Generation 20 done...
Generation 30 done...
Generation 40 done...
Generation 50 done...
Generation 60 done...
Generation 70 done...
Generation 80 done...
Generation 90 done...
Generation 100 done...


In [193]:
len(population[0])

168

In [197]:
history = pd.DataFrame(
    {
        "generation": np.arange(1, len(mean_values) + 1),
        "min": min_values,
        "mean": mean_values,
        "max": max_values,
    }
).melt(id_vars="generation")

fig = px.line(data_frame=history, x="generation", y="value", color="variable")
fig.show()

### Implementing a Clean Version and Making Visualisations

Since the actual structure of the required individuals in genetic algorithms does strongly depend on the task at hand, DEAP does not contain any explicit structure. It will rather provide a convenient method for creating containers of attributes, associated with fitnesses, called the deap.creator. Using this method we can create custom individuals in a very simple way.

In [196]:
# Set fixed variables
TOTAL_GENERATIONS = 80
INDIVIDUAL_SIZE = 2

config = {'CXPB': 0.25,
 'MUTPB': 0.05,
 'MUT_SD': 0.6,
 'MUT_IND_PB': 0.5,
 'TOURNAMENT_SIZE': 3,
 'POP_SIZE': 500}

# Objective Direction
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

# Invidiaul Structure
creator.create("Individual", list, fitness=creator.FitnessMax)

# Initialise the toolbox
toolbox = base.Toolbox()

# Attribute generator for individual genes
toolbox.register("attr_action", np.random.choice, [-1, 0, 1])

# Structure initializers
toolbox.register(
    "individual",
    tools.initRepeat,
    creator.Individual,
    toolbox.attr_action,
    INDIVIDUAL_SIZE,
)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Genetic Operators
toolbox.register("evaluate", evalOneMin)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register(
    "mutate",
    tools.mutUniformInt, 
    low=-1, 
    up=1,
    indpb=config["MUT_IND_PB"],
)

toolbox.register("select", tools.selTournament, tournsize=config["TOURNAMENT_SIZE"])

# Creating the initial
population = toolbox.population(n=config["POP_SIZE"])

# Begin the evolution
generations = []
best_fitnesses = []

for generation in tqdm(range(TOTAL_GENERATIONS)):

    # Select the next generation individuals
    offspring = toolbox.select(population, len(population))

    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if np.random.random() < config["CXPB"]:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if np.random.random() < config["MUTPB"]:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    # Those are the ones that have been deleted
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # Overwrite the current population with the offspring
    population[:] = offspring

    # Increment counter
    generation += 1

    # Get fitnesses
    fitnesses = [ind.fitness.values[0] for ind in population]
    fit_avg = np.mean(fitnesses)
    fit_best = np.min(fitnesses)

    # Get the best individual
    best_index = fitnesses.index(np.min(fitnesses))
    best_indiv = population[best_index]

    # Append the plot information
    generations.append(generation)
    best_fitnesses.append(fit_best)

    # Write the contour plot
    fig = go.Figure(data=[go.Contour(z=Z_ackley, x=x_vals, y=y_vals, 
                                     colorscale="Blues")])
    fig.add_trace(
        go.Scatter(
            x=[ind[0] for ind in population],
            y=[ind[1] for ind in population],
            mode="markers",
            marker=dict(size=3, color="black"),
        )
    )
    fig.add_trace(
        go.Scatter(
            x=[best_indiv[0]],
            y=[best_indiv[1]],
            mode="markers",
            line=dict(width=1, color="black"),
            marker=dict(size=8, color="firebrick"),
        )
    )
    fig.update_layout(
        width=600,
        height=600,
        showlegend=False
    )
    fig.update_xaxes(range=[-5, 5])
    fig.update_yaxes(range=[-5, 5])

    fig.write_image(f"ackley/{generation:04d}.png", scale=2)
    
    # Write the line plot
    fig = px.line(x=generations, y=best_fitnesses)
    fig.update_xaxes(range=[0, TOTAL_GENERATIONS])
    fig.update_layout(
            title=f"Generation: {generation} Best: {fit_best:.2f}",
            title_font=dict(size=28),
            title_x=0.5,
            xaxis_title="Generation",
            yaxis_title="Best Solution",
            width=1200,
            height=300,
            showlegend=False
        )

    fig.write_image(f"ackley_history/{generation:04d}.png", scale=2)

c:\Users\mathi\miniconda3\envs\general\Lib\site-packages\deap\creator.py:185: RuntimeWarning:

A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.

c:\Users\mathi\miniconda3\envs\general\Lib\site-packages\deap\creator.py:185: RuntimeWarning:

A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.



  0%|          | 0/80 [00:00<?, ?it/s]

NameError: name 'ackley' is not defined

### Create a GIF from the PNGs

In [ ]:
def generate_gif(folder_path, output_path):
    # Get list of PNG files sorted
    file_names = sorted([f for f in os.listdir(folder_path) if f.endswith('.png')])

    # Duration of each frame in seconds (assuming 20 frames per second)
    frame_duration = 1 / 20

    # List to store images
    images = []

    writer = imageio.get_writer(output_path, fps=10)
    # Read PNG files and append to images list
    for file_name in file_names:
        file_path = os.path.join(folder_path, file_name)
        im = imageio.imread(file_path)
        writer.append_data(im)

    writer.close()

In [ ]:
# Ackley contour plot
generate_gif("ackley", "ackley_contour.mp4")

# Ackley history
generate_gif("ackley_history", "ackley_history.mp4")


C:\Users\mathi\AppData\Local\Temp\ipykernel_15156\139982273.py:15: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (2400, 600) to (2400, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
